# Tugas Besar IF3170
# Inteligensia Buatan 2017/2018

# Eksperimen untuk mendapatkan model terbaik

## Anggota Kelompok

- 13515019 - Candra Hesen Parera
- 13515024 - Abdurrahman
- 13515059 - Muthmainnah
- 13515090 - Annisa Muzdalifa
- 13515129 - Annisa Nurul Azhar

# Baca Data

In [10]:
from time import time
from sklearn import preprocessing
from sklearn import metrics
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import maxabs_scale
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.externals import joblib
import pandas as pd
import bisect
import numpy as np

with open('CensusIncome.names.txt', 'r') as fname:
    sname = fname.read()
names=np.array(sname[sname.find("age"):].split(".\n"))
#print(names)

#Encode Data Workclass
workclass = preprocessing.LabelEncoder()
workclass.fit(np.array(names[1][names[1].find(":")+2:].split(", ")))
#Handle Unknown Data : '?'
le_classes = workclass.classes_.tolist()
bisect.insort_left(le_classes, '?')
workclass.classes_ = le_classes
#print(workclass.classes_)

#Encode Data Education
education = preprocessing.LabelEncoder()
education.fit(np.array(names[3][names[3].find(":")+2:].split(", ")))
le_classes = education.classes_.tolist()
bisect.insort_left(le_classes, '?')
education.classes_ = le_classes
#print(education.classes_)

marital_status = preprocessing.LabelEncoder()
marital_status.fit(np.array(names[5][names[5].find(":")+2:].split(", ")))
le_classes = marital_status.classes_.tolist()
bisect.insort_left(le_classes, '?')
marital_status.classes_ = le_classes
#print(marital_status.classes_)

occupation = preprocessing.LabelEncoder()
occupation.fit(np.array(names[6][names[6].find(":")+2:].split(", ")))
le_classes = occupation.classes_.tolist()
bisect.insort_left(le_classes, '?')
occupation.classes_ = le_classes
#print(occupation.classes_)

relationship = preprocessing.LabelEncoder()
relationship.fit(np.array(names[7][names[7].find(":")+2:].split(", ")))
le_classes = relationship.classes_.tolist()
bisect.insort_left(le_classes, '?')
relationship.classes_ = le_classes
#print(relationship.classes_)

race = preprocessing.LabelEncoder()
race.fit(np.array(names[8][names[8].find(":")+2:].split(", ")))
le_classes = race.classes_.tolist()
bisect.insort_left(le_classes, '?')
race.classes_ = le_classes
#print(race.classes_)

sex = preprocessing.LabelEncoder()
sex.fit(np.array(names[9][names[9].find(":")+2:].split(", ")))
le_classes = sex.classes_.tolist()
bisect.insort_left(le_classes, '?')
sex.classes_ = le_classes
#print(sex.classes_)

native_country = preprocessing.LabelEncoder()
native_country.fit(np.array(names[13][names[13].find(":")+2:].split(", ")))
le_classes = native_country.classes_.tolist()
bisect.insort_left(le_classes, '?')
native_country.classes_ = le_classes
#print(native_country.classes_)

#Open Data Cencus
with open('CencusIncome.data.txt', 'r') as fdata:
    s = fdata.read()

raw = s.split("\n")
A = []
#print(raw[32560])
length = 32560
for i in range(length):
    A.append(raw[i].split(", "))
    
#Mapping from raw data to  
data_length = 14
cencus_data = []
cencus_target = []
for i in range(length):
    temp = []
    for j in range(data_length):
        if j == 7:
            temp.append(relationship.transform([A[i][j]])[0])
        elif j == 1:
            temp.append(workclass.transform([A[i][j]])[0])
        elif j == 3:
            temp.append(education.transform([A[i][j]])[0])
        elif j == 5:
            temp.append(marital_status.transform([A[i][j]])[0])
        elif j == 6:
            temp.append(occupation.transform([A[i][j]])[0])
        elif j == 8:
            temp.append(race.transform([A[i][j]])[0])
        elif j == 9:
            temp.append(sex.transform([A[i][j]])[0]) 
        elif j == 13:
            temp.append(native_country.transform([A[i][j]])[0])
        else:
            temp.append(int(A[i][j]))
    cencus_data.append(temp)
    cencus_target.append(A[i][data_length])

#Encode Target
target = preprocessing.LabelEncoder()
target.fit(cencus_target)

#Ready to Use Data
y = target.transform(cencus_target)    
X = np.array(cencus_data)

#Scaling
y = maxabs_scale(y, axis=0, copy=False)
X = maxabs_scale(X, axis=0, copy=False)


# k-Nearest Neighbour

In [11]:
kf = KFold(n_splits=10, shuffle=True)

table = []
training_times = []
prediction_times = []
scores = []
i = 0
for train, test in kf.split(X, y):
    x_train = [X[i] for i in train]
    y_train = [y[i] for i in train]
    x_test = [X[i] for i in test]
    y_test = [y[i] for i in test]
    
    t0 = time()
    model = KNeighborsClassifier(n_neighbors=41).fit(x_train, y_train)
    training_time = '{:.6}'.format(time() - t0)
    
    t0 = time()
    pred = model.predict(x_test)
    prediction_time = '{:.6}'.format(time() - t0)
    
    score = accuracy_score(pred, y_test)
    cm = confusion_matrix(y_test, pred)
    i = i+1
    print ("Confusion Matrix Iterasi", i)
    display(pd.DataFrame(confusion_matrix(y_test, pred), columns=['>50K','<=50K'], index=['>50K','<=50K']))
    
    table += [[training_time, prediction_time, score]]
    training_times += [training_time]
    prediction_times += [prediction_time]
    scores += [score]

display(pd.DataFrame(table, columns=['training time', 'prediction time', 'score'], index=range(1,len(table)+1)))

Confusion Matrix Iterasi 1


,>50K,<=50K
>50K,2258,192
<=50K,344,462


Confusion Matrix Iterasi 2


,>50K,<=50K
>50K,2270,210
<=50K,305,471


Confusion Matrix Iterasi 3


...

Confusion Matrix Iterasi 9


,>50K,<=50K
>50K,2280,197
<=50K,336,443


Confusion Matrix Iterasi 10


,>50K,<=50K
>50K,2278,186
<=50K,354,438


,training time,prediction time,score
1,5.81615,7.21522,0.835381
2,6.011,7.37459,0.841830
3,5.35564,5.95662,0.834152
4,5.27074,6.32742,0.836302
5,5.31715,6.54967,0.829545
6,5.2575,5.94056,0.829853
7,5.34577,6.53473,0.839988
8,6.94824,6.58687,0.815418
9,7.11241,6.08218,0.836302
10,5.48975,7.09085,0.834152


# Naive Bayes

In [12]:
#older vers: kf = StratifiedKFold(y, n_folds=10, shuffle=True)
#newer vers: kf = KFold(n_splits=10, shuffle=True)
kf = KFold(n_splits=10, shuffle=True)

table = []
sum = 0
i = 1
for train, test in kf.split(X,y):
    x_train = [X[i] for i in train]
    y_train = [y[i] for i in train]
    x_test = [X[i] for i in test]
    y_test = [y[i] for i in test]
    
    ts = time()
    model = GaussianNB().fit(x_train, y_train)
    trainingtime = time() - ts
    
    ts = time()
    prediction = model.predict(x_test)
    predictiontime = time() - ts
    
    score = accuracy_score(prediction, y_test)
    sum += score
    
    print(i)
    i += 1
    print(metrics.confusion_matrix(y_test,prediction))
    
    table += [[score, trainingtime, predictiontime]]
    
print()
print(pd.DataFrame(table, columns=['score', 'training time', 'prediction time'], index=range(1,len(table)+1)))
print()
print("rata-rata score: ", sum/10)

1
[[2384  119]
 [ 511  242]]
2
[[2407  108]
 [ 479  262]]
3
[[2293  130]
 [ 544  289]]
4
[[2339  145]
 [ 508  264]]
5
[[2358  106]
 [ 527  265]]
6
[[2377  110]
 [ 490  279]]
7
[[2339  133]
 [ 517  267]]
8
[[2350  110]
 [ 544  252]]
9
[[2340  115]
 [ 525  276]]
10
[[2331  126]
 [ 531  268]]

       score  training time  prediction time
1   0.806511       0.136045         0.008433
2   0.819717       0.093756         0.000000
3   0.792998       0.093763         0.023381
4   0.799447       0.093764         0.022708
5   0.805590       0.093758         0.022759
6   0.815725       0.106853         0.008009
7   0.800369       0.096437         0.013395
8   0.799140       0.108038         0.008006
9   0.803440       0.093750         0.000000
10  0.798219       0.105257         0.009378

rata-rata score:  0.804115479115


# Decision Tree Learning

In [13]:
kf = KFold(n_splits=10, shuffle=True)

table = []
training_times = []
prediction_times = []
scores = []
i = 0
for train, test in kf.split(X,y):
    x_train = [X[i] for i in train]
    y_train = [y[i] for i in train]
    x_test = [X[i] for i in test]
    y_test = [y[i] for i in test]
    
    t0 = time()
    model = tree.DecisionTreeClassifier().fit(x_train, y_train)
    training_time = '{:.6}'.format(time() - t0)
    
    t0 = time()
    y_pred = model.predict(x_test)
    prediction_time = '{:.6}'.format(time() - t0)

    score = accuracy_score(y_pred, y_test)
  
    cm = confusion_matrix(y_test, y_pred)
    i = i+1
    print("Confusion Matrix Iterasi =", i)
    display(pd.DataFrame(cm, columns=['>50K','<=50K'], index=['>50K','<=50K']))
    
    table += [[training_time, prediction_time, score]]
    training_times += [training_time]
    prediction_times += [prediction_time]
    scores += [score]
    
display(pd.DataFrame(table, columns=['training time', 'prediction time', 'score'], index=range(1,len(table)+1)))

Confusion Matrix Iterasi = 1


,>50K,<=50K
>50K,2135,317
<=50K,305,499


Confusion Matrix Iterasi = 2


,>50K,<=50K
>50K,2116,327
<=50K,311,502


Confusion Matrix Iterasi = 3


...

Confusion Matrix Iterasi = 9


,>50K,<=50K
>50K,2144,339
<=50K,295,478


Confusion Matrix Iterasi = 10


,>50K,<=50K
>50K,2150,330
<=50K,276,500


,training time,prediction time,score
1,0.59224,0.0,0.808968
2,0.724233,0.00800562,0.804054
3,0.483966,0.0156252,0.812039
4,0.466643,0.0156298,0.804054
5,0.453146,0.0,0.813882
6,0.468774,0.0,0.812654
7,0.468773,0.0,0.819103
8,0.462696,0.015626,0.809889
9,0.469787,0.0156262,0.805283
10,0.486281,0.0156279,0.813882


# Multi Layered Perceptron

In [14]:
mlp = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(100,), random_state=1)
kf = KFold(n_splits=10, shuffle=True)

table = []
training_times = []
prediction_times = []
scores = []
i = 0

for train, test in kf.split(X, y):
    x_train = [X[i] for i in train]
    y_train = [y[i] for i in train]
    x_test = [X[i] for i in test]
    y_test = [y[i] for i in test]
    
    #Xcoba = maxabs_scale(X, axis=0, copy=True)
    x_train = maxabs_scale(x_train, axis=0, copy=False)
    y_train = maxabs_scale(y_train, axis=0, copy=False)
    x_test = maxabs_scale(x_test, axis=0, copy=False)
    y_test = maxabs_scale(y_test, axis=0, copy=False)
    
    t0 = time()
    mlp.fit(x_train, y_train)
    training_time = '{:.6}'.format(time() - t0)
    
    t0 = time()
    pred = mlp.predict(x_test)
    prediction_time = '{:.6}'.format(time() - t0)
    
    score = accuracy_score(pred, y_test)
    cm = confusion_matrix(y_test, pred)
    i = i+1
    print ("Confusion Matrix Iterasi", i)
    display(pd.DataFrame(confusion_matrix(y_test, pred), columns=['>50K','<=50K'], index=['>50K','<=50K']))
    
    table += [[training_time, prediction_time, score]]
    training_times += [training_time]
    prediction_times += [prediction_time]
    scores += [score]

display(pd.DataFrame(table, columns=['training time', 'prediction time', 'score'], index=range(1,len(table)+1)))

Confusion Matrix Iterasi 1


,>50K,<=50K
>50K,2261,183
<=50K,329,483


Confusion Matrix Iterasi 2


,>50K,<=50K
>50K,2308,153
<=50K,351,444


Confusion Matrix Iterasi 3


...

Confusion Matrix Iterasi 9


,>50K,<=50K
>50K,2329,161
<=50K,316,450


Confusion Matrix Iterasi 10


,>50K,<=50K
>50K,2313,194
<=50K,293,456


,training time,prediction time,score
1,52.5429,0.0236473,0.842752
2,51.81,0.0156291,0.845209
3,47.8296,0.0,0.839988
4,48.8112,0.0,0.852580
5,47.3144,0.0156269,0.848280
6,46.0249,0.0156269,0.849201
7,46.5603,0.0,0.844595
8,44.6117,0.0156283,0.846437
9,48.3991,0.01563,0.853501
10,48.8188,0.0156257,0.850430


# Baca Test

In [15]:
#Open Test
with open('CencusIncome.test.txt', 'r') as fdata:
    stest = fdata.read()
    
test_raw = np.array(stest[stest.find("\n")+1:].split(".\n"))
B = []
length = len(test_raw)-1
for i in range(length):
    B.append(test_raw[i].split(", "))

#Mapping from raw data to  
data_length = 14
cencus_data = []
cencus_target = []
for i in range(length):
    temp = []
    for j in range(data_length):
        if j == 7:
            temp.append(relationship.transform([B[i][j]])[0])
        elif j == 1:
            temp.append(workclass.transform([B[i][j]])[0])
        elif j == 3:
            temp.append(education.transform([B[i][j]])[0])
        elif j == 5:
            temp.append(marital_status.transform([B[i][j]])[0])
        elif j == 6:
            temp.append(occupation.transform([B[i][j]])[0])
        elif j == 8:
            temp.append(race.transform([B[i][j]])[0])
        elif j == 9:
            temp.append(sex.transform([B[i][j]])[0]) 
        elif j == 13:
            temp.append(native_country.transform([B[i][j]])[0])
        else:
            temp.append(int(B[i][j]))
    cencus_data.append(temp)
    cencus_target.append(B[i][data_length])

#Encode Target
target = preprocessing.LabelEncoder()
target.fit(cencus_target)

#Ready to Use Data
y_test = target.transform(cencus_target)    
X_test = np.array(cencus_data)

#Scaling
y_test = maxabs_scale(y_test, axis=0, copy=False)
X_test = maxabs_scale(X_test, axis=0, copy=False)

# Save, load, dan predict using best model

In [16]:
model = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(100,), random_state=1)
model.fit(X,y)
joblib.dump(model, 'Best.model')

bestmodel = joblib.load('Best.model')
prediction = bestmodel.predict(X_test)

display(pd.DataFrame(confusion_matrix(y_test, prediction), columns=['>50K','<=50K'], index=['>50K','<=50K']))

,>50K,<=50K
>50K,11588,847
<=50K,1618,2228


In [17]:
print("Score: ", accuracy_score(prediction, y_test))

Score:  0.848596523555
